In [2]:
import pandas as pd
import seaborn as sns
import matplotlib as plt
import os
%matplotlib inline

In [9]:
os.getcwd()

'C:\\Users\\Ришар\\1. Karpov_Courses\\Lesson 5\\Add_project_5'

In [42]:
# Прочитаем все файлы sales
df = pd.DataFrame()
for path,dirs,files in os.walk('C:\\Users\\Ришар\\1. Karpov_Courses\\Lesson 5\\Add_project_5\\tm_sales_for_add_project'):
    for file in files:
        full_path = f'{path}/{files[0]}' 
        df_1 = pd.read_csv(full_path, sep = ';')
        df = pd.concat([df,df_1], ignore_index = True)
df

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25
3,id4651830,1,1954,20-03-2020 14:59
4,7646509,5,6431,19-03-2020 13:00
5,id7412683,4,3313,22-03-2020 17:25
6,id4651830,1,1954,20-03-2020 14:59
7,7646509,5,6431,19-03-2020 13:00
8,id7412683,4,3313,22-03-2020 17:25


In [28]:
df.dtypes

SUBS_ID      object
FILIAL_ID     int64
PROD_ID       int64
ACT_DTTM     object
dtype: object

In [43]:
# если в файле с продажами встречается строка без указанного SUBS_ID, она пропускается
df = df.dropna(subset=['SUBS_ID'])

# добавляем айди ко всем строкам, где их нет
df['SUBS_ID'] = df['SUBS_ID'].apply(lambda x: 'id'+ x if (x.startswith('id') == False) else x)
df

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,id7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25
3,id4651830,1,1954,20-03-2020 14:59
4,id7646509,5,6431,19-03-2020 13:00
5,id7412683,4,3313,22-03-2020 17:25
6,id4651830,1,1954,20-03-2020 14:59
7,id7646509,5,6431,19-03-2020 13:00
8,id7412683,4,3313,22-03-2020 17:25


In [74]:
logs = pd.read_csv('C:\\Users\\Ришар\\1. Karpov_Courses\\Lesson 5\\Add_project_5\\prod_activations_logs.csv', sep = ';')
logs

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM
1,id4651830,1954,20-03-2020 14:59,01-12-2020 00:00
5,id7646509,6431,19-03-2020 13:00,19-03-2020 13:03
2,id7461794,3310,20-03-2020 17:25,01-12-2020 00:00
3,id5416547,1743,17-03-2020 10:17,25-03-2020 11:00
4,id8238421,1859,01-03-2020 11:42,01-03-2020 11:43
3,id8641743,2752,20-03-2020 15:44,21-04-2020 15:44
2,id2185490,3210,16-03-2020 16:28,01-12-2020 00:00
6,id7642700,3020,15-03-2020 14:21,15-03-2020 23:42
2,id8741631,5677,19-03-2020 12:28,01-12-2020 00:00


In [31]:
logs.dtypes

SUBS_ID       object
PROD_ID        int64
START_DTTM    object
END_DTTM      object
dtype: object

In [75]:
# переводим в формат даты и времени
logs.START_DTTM = pd.to_datetime(logs.START_DTTM, format = '%d-%m-%Y %H:%M')
logs.END_DTTM = pd.to_datetime(logs.END_DTTM, format = '%d-%m-%Y %H:%M')

In [77]:
# продажа не засчитывается, если отключение (END_DTTM) произошло меньше чем через 5 минут после подключения (START_DTTM)
# смотрим время выключения
logs['shutdown'] = logs.END_DTTM - logs.START_DTTM
logs

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM,shutdown
1,id4651830,1954,2020-03-20 14:59:00,2020-12-01 00:00:00,255 days 09:01:00
5,id7646509,6431,2020-03-19 13:00:00,2020-03-19 13:03:00,0 days 00:03:00
2,id7461794,3310,2020-03-20 17:25:00,2020-12-01 00:00:00,255 days 06:35:00
3,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00,8 days 00:43:00
4,id8238421,1859,2020-03-01 11:42:00,2020-03-01 11:43:00,0 days 00:01:00
3,id8641743,2752,2020-03-20 15:44:00,2020-04-21 15:44:00,32 days 00:00:00
2,id2185490,3210,2020-03-16 16:28:00,2020-12-01 00:00:00,259 days 07:32:00
6,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00,0 days 09:21:00
2,id8741631,5677,2020-03-19 12:28:00,2020-12-01 00:00:00,256 days 11:32:00


In [55]:
pd.Timedelta('5 min')

Timedelta('0 days 00:05:00')

In [78]:
logs = logs[(logs.shutdown > '5m')]
logs

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM,shutdown
1,id4651830,1954,2020-03-20 14:59:00,2020-12-01 00:00:00,255 days 09:01:00
2,id7461794,3310,2020-03-20 17:25:00,2020-12-01 00:00:00,255 days 06:35:00
3,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00,8 days 00:43:00
3,id8641743,2752,2020-03-20 15:44:00,2020-04-21 15:44:00,32 days 00:00:00
2,id2185490,3210,2020-03-16 16:28:00,2020-12-01 00:00:00,259 days 07:32:00
6,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00,0 days 09:21:00
2,id8741631,5677,2020-03-19 12:28:00,2020-12-01 00:00:00,256 days 11:32:00


In [73]:
# inner merge его с датафрэймом со всеми продажами по колонкам SUBS_ID и PROD_ID
sales_and_logs = logs.merge(df, how = 'inner', on = ['SUBS_ID','PROD_ID'])
sales_and_logs

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM,shutdown,FILIAL_ID,ACT_DTTM
0,id4651830,1954,2020-03-20 14:59:00,2020-12-01,255 days 09:01:00,1,20-03-2020 14:59
1,id4651830,1954,2020-03-20 14:59:00,2020-12-01,255 days 09:01:00,1,20-03-2020 14:59
2,id4651830,1954,2020-03-20 14:59:00,2020-12-01,255 days 09:01:00,1,20-03-2020 14:59
